### Обновление вчера

In [7]:
# =========================
# ЯЧЕЙКА 1 — МОДУЛЬ B: подключение + создание таблицы разметки участков
# =========================
# Цель:
# 1) Подключиться к PostgreSQL через .env (как на соревнованиях).
# 2) Проверить, что базовые таблицы есть.
# 3) Создать таблицу segment_labels — сюда будем сохранять разметку участков:
#    - segment_index (участок внутри трека)
#    - cluster_id (кластер участка)
#    - risk_label (flood/fire/none)
#    - evac_label (easy/medium/hard)
#
# Важно:
# - На соревнованиях меняешь ТОЛЬКО .env
# - Код трогать не надо

import sys
from pathlib import Path

import pandas as pd
from sqlalchemy import text

# ---------------------------------------------------------
# 0) ГАРАНТИРУЕМ, что Python видит пакет src/
#    (ищем корень проекта — где есть папка src)
# ---------------------------------------------------------
def find_project_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):  # обычно достаточно 2-3 уровней вверх, но возьмем запас
        if (cur / "src").exists() and (cur / "src" / "common" / "db.py").exists():
            return cur
        cur = cur.parent
    # если не нашли — вернем стартовую папку (и тогда импорт может упасть)
    return start.resolve()

PROJECT_ROOT = find_project_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(" PROJECT_ROOT =", PROJECT_ROOT)

# ---------------------------------------------------------
# 1) Импортируем наш модуль подключения к БД
# ---------------------------------------------------------
from src.common.db import DBConfig, get_connection, set_search_path, test_connection

# ---------------------------------------------------------
# 2) Проверяем подключение
# ---------------------------------------------------------
test_connection(verbose=True)
cfg = DBConfig.from_env()
print(f"✅ Используем схему: {cfg.schema}")

# ---------------------------------------------------------
# 3) Утилита чтения DataFrame из БД
# ---------------------------------------------------------
def read_df(sql: str, params: dict | None = None) -> pd.DataFrame:
    with get_connection() as conn:
        set_search_path(conn)
        return pd.read_sql(text(sql), conn, params=params)

# ---------------------------------------------------------
# 4) Проверяем наличие таблиц (быстро убеждаемся, что база готова)
# ---------------------------------------------------------
tables = read_df("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = :schema
ORDER BY table_name;
""", {"schema": cfg.schema})

print(" Таблицы в схеме:")
display(tables)

# ---------------------------------------------------------
# 5) Создаём таблицу разметки участков (segment_labels)
# ---------------------------------------------------------
create_sql = f"""
CREATE TABLE IF NOT EXISTS "{cfg.schema}"."segment_labels" (
    segment_id      BIGSERIAL PRIMARY KEY,

    track_id        BIGINT NOT NULL,
    segment_index   INT NOT NULL,               -- номер участка внутри трека (0,1,2,...)

    start_seq       INT NULL,                   -- стартовая точка (seq) внутри трека
    end_seq         INT NULL,                   -- конечная точка (seq) внутри трека

    lat_center      DOUBLE PRECISION NULL,
    lon_center      DOUBLE PRECISION NULL,

    features        JSONB NOT NULL DEFAULT '{{}}'::jsonb,  -- фичи участка (что посчитали)
    cluster_id      INT NULL,                                -- кластер участка

    risk_label      TEXT NULL,                 -- flood / fire / none
    evac_label      TEXT NULL,                 -- easy / medium / hard

    created_at      TIMESTAMPTZ NOT NULL DEFAULT NOW(),

    UNIQUE(track_id, segment_index)
);
"""

with get_connection() as conn:
    set_search_path(conn)
    conn.execute(text(create_sql))
    conn.commit()

print(" Таблица segment_labels готова (или уже существовала).")

# ---------------------------------------------------------
# 6) Мини-проверка: таблица реально доступна
# ---------------------------------------------------------
check = read_df("""SELECT COUNT(*) AS rows_cnt FROM segment_labels;""")
print(" segment_labels сейчас строк:", int(check.loc[0, "rows_cnt"]))
display(check)

 PROJECT_ROOT = /Users/iriska/Downloads/mlbox_2
OK: connected to PostgreSQL | host=localhost port=55432 db=gpx_db schema=public
✅ Используем схему: public
 Таблицы в схеме:


,table_name
0,cluster_runs
1,context_time_series
2,data_sources
3,model_runs
4,segment_labels
5,track_cluster_labels
6,track_features
7,track_points
8,track_predictions
9,tracks


 Таблица segment_labels готова (или уже существовала).
 segment_labels сейчас строк: 0


,rows_cnt
0,0


In [8]:
# =========================
# ЯЧЕЙКА 2 — МОДУЛЬ B: нарезка треков на участки + запись в segment_labels
# =========================
# Идея простая:
# 1) Берём точки track_points для каждого трека
# 2) Режем на сегменты по SEGMENT_SIZE точек (например, по 300)
# 3) Считаем простые признаки сегмента (длина, набор высот, скорость, уклон и т.д.)
# 4) Сохраняем сегменты в segment_labels (UPSERT: если сегмент уже есть — обновим)

import json
import numpy as np
import pandas as pd
from sqlalchemy import text

# 🔧 Настройки (на соревнованиях можно быстро менять)
SEGMENT_SIZE = 10    # 100/200/300/500 — регулирует "детальность" сегментов
LIMIT_TRACKS = None   # None = все треки, или 5 для быстрой проверки

# -------------------------
# 1) Достаём точки трека
# -------------------------
def fetch_track_points(track_id: int) -> pd.DataFrame:
    """
    ВАЖНО:
    - В БД колонка высоты называется ele (а не ele_m)
    - Мы делаем alias: ele AS ele_m, чтобы дальше в коде везде было единообразно 'ele_m'
    """
    sql = """
    SELECT
        track_id,
        seq,
        lat,
        lon,
        ele AS ele_m,
        time
    FROM track_points
    WHERE track_id = :track_id
    ORDER BY seq;
    """
    return read_df(sql, {"track_id": track_id})

# -------------------------
# 2) Гео-расстояние (haversine)
# -------------------------
def haversine_m(lat1, lon1, lat2, lon2) -> float:
    """Расстояние между двумя координатами в метрах."""
    R = 6371000.0
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dl = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dl/2)**2
    return float(2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))

# -------------------------
# 3) Признаки сегмента
# -------------------------
def calc_segment_features(df_seg: pd.DataFrame) -> dict:
    """
    Признаки сегмента (универсально):
    - distance_m: длина сегмента по прямой (между первой и последней точкой)
    - elev_*: минимум/максимум/набор/сброс высоты (если есть)
    - duration_s и avg_speed_mps (если есть time)
    - slope: грубый уклон (перепад/дистанция)
    - points: кол-во точек в сегменте
    """

    lat0, lon0 = df_seg.iloc[0][["lat", "lon"]]
    lat1, lon1 = df_seg.iloc[-1][["lat", "lon"]]
    dist_m = haversine_m(lat0, lon0, lat1, lon1)

    # высоты (могут отсутствовать)
    ele = df_seg["ele_m"].dropna()
    if len(ele) > 1:
        d = ele.diff()
        elev_gain = float(d.clip(lower=0).sum())
        elev_loss = float((-d.clip(upper=0)).sum())
    else:
        elev_gain, elev_loss = None, None

    elev_min = float(ele.min()) if len(ele) else None
    elev_max = float(ele.max()) if len(ele) else None

    # время/скорость (в некоторых треках time может быть NaT)
    if df_seg["time"].notna().any():
        t0 = df_seg["time"].dropna().iloc[0]
        t1 = df_seg["time"].dropna().iloc[-1]
        duration_s = float((t1 - t0).total_seconds()) if (t0 is not None and t1 is not None) else None
    else:
        duration_s = None

    avg_speed_mps = (dist_m / duration_s) if (duration_s and duration_s > 0) else None

    slope = None
    if elev_min is not None and elev_max is not None and dist_m > 0:
        slope = float((elev_max - elev_min) / dist_m)

    return {
        "distance_m": dist_m,
        "duration_s": duration_s,
        "avg_speed_mps": avg_speed_mps,
        "elev_gain_m": elev_gain,
        "elev_loss_m": elev_loss,
        "elev_min_m": elev_min,
        "elev_max_m": elev_max,
        "slope": slope,
        "points": int(len(df_seg)),
    }

# -------------------------
# 4) UPSERT сегмента в segment_labels
# -------------------------
def upsert_segment_row(track_id: int, segment_index: int, start_seq: int, end_seq: int,
                       lat_center: float, lon_center: float, features: dict) -> None:
    """
    UPSERT:
    - если сегмент (track_id, segment_index) уже есть — обновляем
    - если нет — вставляем
    """
    sql = f"""
    INSERT INTO "{cfg.schema}"."segment_labels"
        (track_id, segment_index, start_seq, end_seq, lat_center, lon_center, features)
    VALUES
        (:track_id, :segment_index, :start_seq, :end_seq, :lat_center, :lon_center, CAST(:features AS jsonb))
    ON CONFLICT (track_id, segment_index)
    DO UPDATE SET
        start_seq = EXCLUDED.start_seq,
        end_seq = EXCLUDED.end_seq,
        lat_center = EXCLUDED.lat_center,
        lon_center = EXCLUDED.lon_center,
        features = EXCLUDED.features;
    """

    features_json = json.dumps(features, ensure_ascii=False)

    with get_connection() as conn:
        set_search_path(conn)
        conn.execute(text(sql), {
            "track_id": track_id,
            "segment_index": segment_index,
            "start_seq": start_seq,
            "end_seq": end_seq,
            "lat_center": lat_center,
            "lon_center": lon_center,
            "features": features_json,
        })
        conn.commit()

# -------------------------
# 5) Запуск нарезки по всем/части треков
# -------------------------
tracks_df = read_df("SELECT track_id, point_count FROM tracks ORDER BY track_id;")
if LIMIT_TRACKS is not None:
    tracks_df = tracks_df.head(LIMIT_TRACKS)

print("Треков для обработки:", len(tracks_df))
print("SEGMENT_SIZE:", SEGMENT_SIZE)

inserted = 0
skipped_empty = 0

for _, row in tracks_df.iterrows():
    track_id = int(row["track_id"])
    pts = fetch_track_points(track_id)

    if pts.empty:
        skipped_empty += 1
        continue

    n_segments = int(np.ceil(len(pts) / SEGMENT_SIZE))

    for seg_idx in range(n_segments):
        start = seg_idx * SEGMENT_SIZE
        end = min((seg_idx + 1) * SEGMENT_SIZE, len(pts))
        df_seg = pts.iloc[start:end].copy()
        if df_seg.empty:
            continue

        start_seq = int(df_seg["seq"].iloc[0])
        end_seq = int(df_seg["seq"].iloc[-1])

        lat_center = float(df_seg["lat"].mean())
        lon_center = float(df_seg["lon"].mean())

        feats = calc_segment_features(df_seg)

        upsert_segment_row(
            track_id=track_id,
            segment_index=seg_idx,
            start_seq=start_seq,
            end_seq=end_seq,
            lat_center=lat_center,
            lon_center=lon_center,
            features=feats
        )
        inserted += 1

print("✅ Готово.")
print("Сегментов записано/обновлено:", inserted)
print("Треков без точек пропущено:", skipped_empty)

# -------------------------
# 6) Контроль: сколько строк в segment_labels + примеры
# -------------------------
check2 = read_df("SELECT COUNT(*) AS rows_cnt FROM segment_labels;")
print("📊 segment_labels теперь строк:", int(check2.loc[0, "rows_cnt"]))
display(check2)

sample = read_df("""
SELECT segment_id, track_id, segment_index, start_seq, end_seq, lat_center, lon_center, features
FROM segment_labels
ORDER BY segment_id
LIMIT 10;
""")
display(sample)

Треков для обработки: 42
SEGMENT_SIZE: 10
✅ Готово.
Сегментов записано/обновлено: 8663
Треков без точек пропущено: 0
📊 segment_labels теперь строк: 8663


,rows_cnt
0,8663


,segment_id,track_id,segment_index,start_seq,end_seq,lat_center,lon_center,features
0,1,2,0,1,10,53.567823,9.896177,"{'slope': 5.646957576726941, 'points': 10, 'di..."
1,2,2,1,11,20,53.569020,9.892310,"{'slope': 0.037576334892397326, 'points': 10, ..."
2,3,2,2,21,30,53.568016,9.901295,"{'slope': 0.07439179690716077, 'points': 10, '..."
3,4,2,3,31,40,53.567848,9.896189,"{'slope': 4.3854984776862445, 'points': 10, 'd..."
4,5,2,4,41,45,53.567841,9.896223,"{'slope': 4.532289263840481, 'points': 5, 'dis..."
5,6,3,0,1,10,46.188408,-119.217538,"{'slope': 0.05858891254906029, 'points': 10, '..."
6,7,3,1,11,20,46.188490,-119.217676,"{'slope': 0.09943825607635966, 'points': 10, '..."
7,8,3,2,21,30,46.188502,-119.217682,"{'slope': 0.042447543877938046, 'points': 10, ..."
8,9,3,3,31,40,46.188450,-119.217815,"{'slope': 0.018022269585323783, 'points': 10, ..."
9,10,3,4,41,50,46.188530,-119.218030,"{'slope': 0.06113833866494918, 'points': 10, '..."


In [9]:
# =========================
# ЯЧЕЙКА 3 — МОДУЛЬ B: кластеризация СЕГМЕНТОВ + качество + запись cluster_id в БД
# =========================
# Что делаем:
# 1) Читаем segment_labels.features (JSON) -> вытаскиваем признаки в DataFrame
# 2) Imputer (median) + StandardScaler
# 3) Пробуем несколько k, считаем silhouette + inertia
# 4) Автоматически предлагаем K_BEST (по silhouette) + выводим таблицу
# 5) Обучаем финальную модель и записываем cluster_id в segment_labels
# 6) Профилирование кластеров (средние значения признаков) — чтобы потом размечать risk/evac

import json
import numpy as np
import pandas as pd

from sqlalchemy import text
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# -------------------------
# 0) Настройки
# -------------------------
K_RANGE = range(2, 11)  # k=2..10 (можешь расширить)
RANDOM_STATE = 42

# Базовый универсальный набор признаков для сегмента
SEG_FEATURES = [
    "distance_m", "duration_s", "avg_speed_mps",
    "elev_gain_m", "elev_loss_m", "elev_min_m", "elev_max_m",
    "slope", "points"
]

# -------------------------
# 1) Читаем сегменты из БД
# -------------------------
seg_raw = read_df("""
SELECT segment_id, track_id, segment_index, features
FROM segment_labels
ORDER BY segment_id;
""")

print("✅ segment_labels rows:", len(seg_raw))
display(seg_raw.head(3))

# -------------------------
# 2) Распаковка JSON features в колонки
# -------------------------
def unpack_features(x) -> dict:
    if x is None:
        return {}
    if isinstance(x, dict):
        return x
    if isinstance(x, str):
        try:
            return json.loads(x)
        except Exception:
            return {}
    return {}

rows = []
for _, r in seg_raw.iterrows():
    f = unpack_features(r["features"])
    row = {
        "segment_id": int(r["segment_id"]),
        "track_id": int(r["track_id"]),
        "segment_index": int(r["segment_index"]),
    }
    for k in SEG_FEATURES:
        row[k] = f.get(k, np.nan)
    rows.append(row)

df_seg = pd.DataFrame(rows)

print("\n✅ df_seg shape:", df_seg.shape)
display(df_seg.head(5))

# -------------------------
# 3) Подготовка X (NaN -> median, потом scaling)
# -------------------------
X = df_seg[SEG_FEATURES].copy()

print("\n📊 NaN по признакам (это нормально — имьютер исправит):")
display(X.isna().sum().to_frame("nan_cnt"))

prep = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

X_prep = prep.fit_transform(X)

# -------------------------
# 4) Подбор k: silhouette + inertia
# -------------------------
results = []
for k in K_RANGE:
    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    labels = km.fit_predict(X_prep)

    # silhouette считается только если кластеров > 1 и не все точки в одном
    sil = silhouette_score(X_prep, labels) if len(set(labels)) > 1 else np.nan

    results.append({"k": k, "silhouette": sil, "inertia": km.inertia_})

df_k = pd.DataFrame(results).sort_values("k")

print("\n📌 Метрики качества кластеризации по k:")
display(df_k)

# Авто-предложение K_BEST: выбираем k с максимальным silhouette
k_best_auto = int(df_k.loc[df_k["silhouette"].idxmax(), "k"])
best_sil = float(df_k["silhouette"].max())

print(f"\n⭐ Авто-рекомендация K_BEST = {k_best_auto} (silhouette={best_sil:.4f})")
print("Если тебе важно 'чтобы было понятно' — обычно берут 3..6 кластеров, даже если silhouette чуть хуже.")

# -------------------------
# 5) Финальная модель (можешь вручную переопределить)
# -------------------------
K_BEST = k_best_auto   # <-- если хочешь вручную: K_BEST = 5

km_final = KMeans(n_clusters=K_BEST, random_state=RANDOM_STATE, n_init=10)
df_seg["cluster_id"] = km_final.fit_predict(X_prep)

print(f"\n✅ Кластеры присвоены (K_BEST={K_BEST}). Пример:")
display(df_seg[["segment_id", "track_id", "segment_index", "cluster_id"]].head(10))

# -------------------------
# 6) Профилирование кластеров (средние значения признаков)
# -------------------------
profile = df_seg.groupby("cluster_id")[SEG_FEATURES].mean(numeric_only=True).round(3)
cnts = df_seg["cluster_id"].value_counts().sort_index().rename("count")
profile = profile.join(cnts)

print("\n🧠 Профили кластеров (средние значения признаков + count):")
display(profile)

# -------------------------
# 7) Запись cluster_id в segment_labels
# -------------------------
update_sql = f"""
UPDATE "{cfg.schema}"."segment_labels"
SET cluster_id = :cluster_id
WHERE segment_id = :segment_id;
"""

with get_connection() as conn:
    set_search_path(conn)
    for _, r in df_seg[["segment_id", "cluster_id"]].iterrows():
        conn.execute(text(update_sql), {
            "segment_id": int(r["segment_id"]),
            "cluster_id": int(r["cluster_id"]),
        })
    conn.commit()

print("✅ cluster_id записаны в segment_labels.")

# -------------------------
# 8) Контроль: распределение кластеров из БД
# -------------------------
dist = read_df("""
SELECT cluster_id, COUNT(*) AS cnt
FROM segment_labels
GROUP BY cluster_id
ORDER BY cluster_id;
""")

print("\n📊 Распределение сегментов по кластерам:")
display(dist)

✅ segment_labels rows: 8663


,segment_id,track_id,segment_index,features
0,1,2,0,"{'slope': 5.646957576726941, 'points': 10, 'di..."
1,2,2,1,"{'slope': 0.037576334892397326, 'points': 10, ..."
2,3,2,2,"{'slope': 0.07439179690716077, 'points': 10, '..."



✅ df_seg shape: (8663, 12)


,segment_id,track_id,segment_index,distance_m,duration_s,avg_speed_mps,elev_gain_m,elev_loss_m,elev_min_m,elev_max_m,slope,points
0,1,2,0,15.008133,15748.442,0.000953,135.983565,110.327036,16.976774,101.727065,5.646958,10
1,2,2,1,1385.158823,28361.071,0.048840,80.556518,94.560507,45.917909,97.967101,0.037576,10
2,3,2,2,764.505079,2961.397,0.258157,70.112958,84.322839,57.971014,114.843920,0.074392,10
3,4,2,3,12.823219,31416.054,0.000408,111.152969,74.992999,49.397907,105.634113,4.385498,10
4,5,2,4,13.362926,2079.781,0.006425,22.271424,60.564648,69.724232,130.288880,4.532289,5



📊 NaN по признакам (это нормально — имьютер исправит):


...

,segment_id,track_id,segment_index,cluster_id
0,1,2,0,4
1,2,2,1,4
2,3,2,2,2
3,4,2,3,4
4,5,2,4,3
5,6,3,0,2
6,7,3,1,2
7,8,3,2,2
8,9,3,3,2
9,10,3,4,2



🧠 Профили кластеров (средние значения признаков + count):


,distance_m,duration_s,avg_speed_mps,elev_gain_m,elev_loss_m,elev_min_m,elev_max_m,slope,points,count
cluster_id,,,,,,,,,,
0,773.484,214.444,4.027,3.168,3.237,181.478,185.567,0.017,9.999,7300
1,574.466,288.205,3.849,10.160,10.529,784.235,798.832,0.040,9.997,1011
2,16.883,22.955,0.861,1.292,1.329,144.363,146.297,0.254,9.991,323
3,324.384,437.756,2.663,3.054,7.147,218.482,224.937,0.478,3.962,26
4,470.997,25175.189,0.017,109.231,93.294,37.431,101.776,3.357,10.000,3


✅ cluster_id записаны в segment_labels.

📊 Распределение сегментов по кластерам:


,cluster_id,cnt
0,0,7300
1,1,1011
2,2,323
3,3,26
4,4,3


In [2]:
from sqlalchemy import text

with get_connection() as conn:
    set_search_path(conn)
    conn.execute(text("""
        UPDATE segment_labels
        SET risk_label = NULL,
            evac_label = NULL
        ;
    """))
    conn.commit()

print("✅ Разметка risk/evac очищена (cluster_id и features остались).")

✅ Разметка risk/evac очищена (cluster_id и features остались).


In [10]:
# =========================
# ЯЧЕЙКА 4 — МОДУЛЬ B: разметка сегментов (risk / evac) + запись в БД
# =========================
# Смысл:
# 1) Берём сегменты из segment_labels (у них уже есть cluster_id и базовые features из ячейки 2).
# 2) Подтягиваем ближайший context_time_series (если есть) по source_id и по координатам.
# 3) Из контекста вытаскиваем:
#    - weather.temp_c (если есть)
#    - nearby.counts (если есть)
# 4) Полуавтоматически ставим метки:
#    - risk_label: none / flood / fire
#    - evac_label: easy / medium / hard
# 5) Записываем обратно в segment_labels.

import json
import numpy as np
import pandas as pd
from sqlalchemy import text

# --- НАСТРОЙКИ (на соревнованиях меняются тут)
MATCH_RADIUS_M = 400   # радиус поиска ближайшего контекста к сегменту
FLOOD_WATERWAY_TH = 3  # если рядом много waterway => "flood"
FIRE_TEMP_TH = 25      # если жарко и много natural/landuse => "fire"
HARD_SLOPE_TH = 0.12   # уклон выше => сложнее эвакуация
MED_SLOPE_TH  = 0.05

# ---------- утилиты ----------
def to_dict(x):
    if x is None:
        return {}
    if isinstance(x, dict):
        return x
    if isinstance(x, str):
        try:
            return json.loads(x)
        except Exception:
            return {}
    return {}

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dl = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dl/2)**2
    return float(2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))

def counts_total(counts_dict, tag):
    # counts_dict[tag] обычно вида: [['footway', 120], ['service', 16], ...]
    arr = counts_dict.get(tag, [])
    s = 0
    for item in arr:
        # item может быть list/tuple: [name, cnt]
        if isinstance(item, (list, tuple)) and len(item) >= 2:
            try:
                s += float(item[1])
            except Exception:
                pass
    return float(s)

# ---------- 1) читаем сегменты ----------
seg = read_df("""
SELECT
  segment_id, track_id, segment_index,
  lat_center, lon_center,
  features, cluster_id
FROM segment_labels
ORDER BY segment_id;
""")
seg["features_dict"] = seg["features"].apply(to_dict)

print("✅ segment_labels:", seg.shape)
display(seg.head(3))

# ---------- 2) добавляем source_id через tracks (чтобы мэтчить с context_time_series) ----------
tracks_map = read_df("SELECT track_id, source_id FROM tracks;")
seg = seg.merge(tracks_map, on="track_id", how="left")

# ---------- 3) читаем контекст ----------
ctx = read_df("""
SELECT context_id, source_id, lat, lon, time, values
FROM context_time_series
ORDER BY context_id;
""")
ctx["values_dict"] = ctx["values"].apply(to_dict)

print("✅ context_time_series:", ctx.shape)
display(ctx.head(3))

# ---------- 4) мэтчим контекст к сегментам (по source_id + ближайшая точка в радиусе) ----------
# Важно: контекста у тебя мало (например 89 строк), поэтому мэтч будет только части сегментов — это нормально.
ctx_by_source = {sid: df.reset_index(drop=True) for sid, df in ctx.groupby("source_id")}

matched_ctx_id = []
matched_ctx_dist = []
matched_ctx_values = []

for _, r in seg.iterrows():
    sid = r["source_id"]
    latc, lonc = float(r["lat_center"]), float(r["lon_center"])

    if sid not in ctx_by_source or pd.isna(sid):
        matched_ctx_id.append(None)
        matched_ctx_dist.append(None)
        matched_ctx_values.append({})
        continue

    dfc = ctx_by_source[sid]
    # считаем расстояния до всех контекст-точек этого source_id
    dists = dfc.apply(lambda x: haversine_m(latc, lonc, float(x["lat"]), float(x["lon"])), axis=1)
    j = int(dists.idxmin())
    dmin = float(dists.loc[j])

    if dmin <= MATCH_RADIUS_M:
        matched_ctx_id.append(int(dfc.loc[j, "context_id"]))
        matched_ctx_dist.append(dmin)
        matched_ctx_values.append(dfc.loc[j, "values_dict"] if isinstance(dfc.loc[j, "values_dict"], dict) else {})
    else:
        matched_ctx_id.append(None)
        matched_ctx_dist.append(dmin)
        matched_ctx_values.append({})

seg["ctx_id"] = matched_ctx_id
seg["ctx_dist_m"] = matched_ctx_dist
seg["ctx_values"] = matched_ctx_values

print(f"🔗 context matched: {seg['ctx_id'].notna().sum()} | no_ctx: {seg['ctx_id'].isna().sum()} | total: {len(seg)}")

# ---------- 5) достаем weather/nearby из ctx_values ----------
def extract_ctx_features(v: dict) -> dict:
    weather = v.get("weather", {}) if isinstance(v.get("weather", {}), dict) else {}
    nearby  = v.get("nearby", {})  if isinstance(v.get("nearby", {}), dict) else {}
    counts  = nearby.get("counts", {}) if isinstance(nearby.get("counts", {}), dict) else {}

    # temp
    temp_c = weather.get("temp_c", np.nan)
    ok = weather.get("ok", False)

    out = {
        "weather_ok": bool(ok),
        "temp_c": float(temp_c) if temp_c is not None and temp_c == temp_c else np.nan,  # temp_c==temp_c проверка на nan
        "nearby_highway_total": counts_total(counts, "highway"),
        "nearby_natural_total": counts_total(counts, "natural"),
        "nearby_landuse_total": counts_total(counts, "landuse"),
        "nearby_waterway_total": counts_total(counts, "waterway"),
        "nearby_building_total": counts_total(counts, "building"),
        "nearby_amenity_total": counts_total(counts, "amenity"),
    }
    return out

ctx_feats = seg["ctx_values"].apply(lambda v: extract_ctx_features(v) if isinstance(v, dict) else {})
ctx_feats_df = pd.json_normalize(ctx_feats)
seg = pd.concat([seg, ctx_feats_df], axis=1)

# ---------- 6) берём базовые сегмент-фичи из features_dict ----------
def getf(fd, k, default=np.nan):
    try:
        v = fd.get(k, default)
        return v
    except Exception:
        return default

seg["slope"]      = seg["features_dict"].apply(lambda d: float(getf(d,"slope", np.nan)) if getf(d,"slope", np.nan) is not None else np.nan)
seg["distance_m"] = seg["features_dict"].apply(lambda d: float(getf(d,"distance_m", np.nan)) if getf(d,"distance_m", np.nan) is not None else np.nan)

# ---------- 7) ПРАВИЛА РАЗМЕТКИ (полуавтоматическая логика) ----------
def assign_risk(row) -> str:
    # Если контекста нет — лучше ставить none (честно), чем выдумывать.
    if pd.isna(row.get("ctx_id", np.nan)):
        return "none"

    waterway = float(row.get("nearby_waterway_total", 0) or 0)
    natural  = float(row.get("nearby_natural_total", 0) or 0)
    landuse  = float(row.get("nearby_landuse_total", 0) or 0)
    temp     = row.get("temp_c", np.nan)

    # flood: много водных объектов рядом
    if waterway >= FLOOD_WATERWAY_TH:
        return "flood"

    # fire: жарко + много природы/растительности
    if (temp == temp) and (temp >= FIRE_TEMP_TH) and ((natural + landuse) >= 20):
        return "fire"

    return "none"

def assign_evac(row) -> str:
    # уклон + доступность дорог рядом
    slope = row.get("slope", np.nan)
    highway = float(row.get("nearby_highway_total", 0) or 0)

    # если контекст есть и дорог много — проще эвакуация
    if highway >= 200:
        return "easy"

    # если уклон неизвестен — ставим medium (нейтрально)
    if not (slope == slope):
        return "medium"

    if slope >= HARD_SLOPE_TH:
        return "hard"
    if slope >= MED_SLOPE_TH:
        return "medium"
    return "easy"

seg["risk_label"] = seg.apply(assign_risk, axis=1)
seg["evac_label"] = seg.apply(assign_evac, axis=1)

# ---------- 8) формируем новый features (добавляем ctx-фичи внутрь json) ----------
def merge_features(fd: dict, row) -> dict:
    base = dict(fd) if isinstance(fd, dict) else {}
    base["ctx"] = {
        "matched": bool(row.get("ctx_id") == row.get("ctx_id")),
        "ctx_id": int(row["ctx_id"]) if row.get("ctx_id") == row.get("ctx_id") else None,
        "ctx_dist_m": float(row["ctx_dist_m"]) if row.get("ctx_dist_m") == row.get("ctx_dist_m") else None,
        "temp_c": float(row["temp_c"]) if row.get("temp_c") == row.get("temp_c") else None,
        "nearby_highway_total": float(row.get("nearby_highway_total", 0) or 0),
        "nearby_natural_total": float(row.get("nearby_natural_total", 0) or 0),
        "nearby_landuse_total": float(row.get("nearby_landuse_total", 0) or 0),
        "nearby_waterway_total": float(row.get("nearby_waterway_total", 0) or 0),
        "nearby_building_total": float(row.get("nearby_building_total", 0) or 0),
        "nearby_amenity_total": float(row.get("nearby_amenity_total", 0) or 0),
    }
    return base

seg["features_new"] = seg.apply(lambda r: merge_features(r["features_dict"], r), axis=1)

# ---------- 9) СТАТИСТИКА (чтобы увидеть, что получилось) ----------
print("\n📊 risk_label распределение:")
display(seg["risk_label"].value_counts(dropna=False).to_frame("cnt"))

print("\n📊 evac_label распределение:")
display(seg["evac_label"].value_counts(dropna=False).to_frame("cnt"))

print("\n📊 matched ctx дистанции (если есть):")
display(seg.loc[seg["ctx_id"].notna(), "ctx_dist_m"].describe().to_frame().T)

print("✅ matched ctx:", int(seg["ctx_id"].notna().sum()), "из", len(seg))
print("✅ share matched %:", round(100 * seg["ctx_id"].notna().mean(), 2))

# ---------- 10) Запись в БД (ВАЖНО: без :features::jsonb, используем CAST) ----------
def to_json_str(x) -> str:
    return json.dumps(x, ensure_ascii=False)

upd_sql = text(f"""
UPDATE "{cfg.schema}"."segment_labels"
SET risk_label = :risk_label,
    evac_label = :evac_label,
    features   = CAST(:features AS jsonb)
WHERE segment_id = :segment_id;
""")

rows_updated = 0
with get_connection() as conn:
    set_search_path(conn)
    for _, rr in seg.iterrows():
        conn.execute(upd_sql, {
            "segment_id": int(rr["segment_id"]),
            "risk_label": str(rr["risk_label"]),
            "evac_label": str(rr["evac_label"]),
            "features": to_json_str(rr["features_new"]),
        })
        rows_updated += 1
    conn.commit()

print(f"\n✅ Записали разметку в БД. Обновлено строк: {rows_updated}")

# ---------- 11) Контрольный просмотр ----------
check = read_df("""
SELECT segment_id, track_id, segment_index, cluster_id, risk_label, evac_label
FROM segment_labels
ORDER BY segment_id
LIMIT 15;
""")
display(check)

✅ segment_labels: (8663, 8)


,segment_id,track_id,segment_index,lat_center,lon_center,features,cluster_id,features_dict
0,1,2,0,53.567823,9.896177,"{'slope': 5.646957576726941, 'points': 10, 'di...",4,"{'slope': 5.646957576726941, 'points': 10, 'di..."
1,2,2,1,53.569020,9.892310,"{'slope': 0.037576334892397326, 'points': 10, ...",4,"{'slope': 0.037576334892397326, 'points': 10, ..."
2,3,2,2,53.568016,9.901295,"{'slope': 0.07439179690716077, 'points': 10, '...",2,"{'slope': 0.07439179690716077, 'points': 10, '..."


✅ context_time_series: (89, 7)


,context_id,source_id,lat,lon,time,values,values_dict
0,1,2,53.567858,9.896249,2026-02-01 23:03:48.984000+00:00,"{'point': {'seq': 1, 'ele_m': 76.0705366265677...","{'point': {'seq': 1, 'ele_m': 76.0705366265677..."
1,2,3,46.188395,-119.217436,2026-02-03 00:45:03.999000+00:00,"{'point': {'seq': 1, 'ele_m': 234.45009, 'segm...","{'point': {'seq': 1, 'ele_m': 234.45009, 'segm..."
2,3,4,38.763706,-121.147258,2026-01-31 22:07:40+00:00,"{'point': {'seq': 1, 'ele_m': 141.800003051757...","{'point': {'seq': 1, 'ele_m': 141.800003051757..."


🔗 context matched: 496 | no_ctx: 8167 | total: 8663

📊 risk_label распределение:


...

,cnt
evac_label,
easy,5133
medium,3267
hard,263



📊 matched ctx дистанции (если есть):


,count,mean,std,min,25%,50%,75%,max
ctx_dist_m,496.0,130.461497,118.507173,3.950437e-10,28.389115,87.597736,219.849732,398.088595


✅ matched ctx: 496 из 8663
✅ share matched %: 5.73

✅ Записали разметку в БД. Обновлено строк: 8663


,segment_id,track_id,segment_index,cluster_id,risk_label,evac_label
0,1,2,0,4,none,hard
1,2,2,1,4,none,easy
2,3,2,2,2,none,medium
3,4,2,3,4,none,hard
4,5,2,4,3,none,hard
5,6,3,0,2,none,medium
6,7,3,1,2,none,medium
7,8,3,2,2,none,easy
8,9,3,3,2,none,easy
9,10,3,4,2,none,medium


In [12]:
sample = read_df("""
SELECT
    segment_id,
    track_id,
    segment_index,
    cluster_id,
    features
FROM segment_labels
LIMIT 10;
""")

display(sample)

,segment_id,track_id,segment_index,cluster_id,features
0,4,2,3,4,"{'ctx': {'ctx_id': 1, 'temp_c': -6.2, 'matched..."
1,5,2,4,3,"{'ctx': {'ctx_id': 1, 'temp_c': -6.2, 'matched..."
2,6,3,0,2,"{'ctx': {'ctx_id': 2, 'temp_c': 8.5, 'matched'..."
3,7,3,1,2,"{'ctx': {'ctx_id': 2, 'temp_c': 8.5, 'matched'..."
4,8,3,2,2,"{'ctx': {'ctx_id': 2, 'temp_c': 8.5, 'matched'..."
5,9,3,3,2,"{'ctx': {'ctx_id': 2, 'temp_c': 8.5, 'matched'..."
6,10,3,4,2,"{'ctx': {'ctx_id': 2, 'temp_c': 8.5, 'matched'..."
7,11,3,5,2,"{'ctx': {'ctx_id': 2, 'temp_c': 8.5, 'matched'..."
8,12,4,0,0,"{'ctx': {'ctx_id': 3, 'temp_c': 14.2, 'matched..."
9,13,4,1,0,"{'ctx': {'ctx_id': 3, 'temp_c': 14.2, 'matched..."


In [13]:
read_df("""
SELECT
    COUNT(*) AS total,
    COUNT(*) FILTER (WHERE (features->>'slope')::float > 0.3) AS steep_cnt,
    COUNT(*) FILTER (WHERE (features->>'slope')::float > 0.6) AS very_steep_cnt
FROM segment_labels;
""")

,total,steep_cnt,very_steep_cnt
0,8663,77,19


In [14]:
read_df("""
SELECT
    COUNT(*) AS total,
    COUNT(*) FILTER (WHERE (features->>'distance_to_water_m') IS NOT NULL) AS with_water
FROM segment_labels;
""")

,total,with_water
0,8663,0


In [6]:
# =========================
# СЛУЖЕБНАЯ ЯЧЕЙКА — ПОЛНАЯ ОЧИСТКА segment_labels
# =========================
# Используем ТОЛЬКО когда хотим пересобрать сегменты с нуля
# (например, меняем SEGMENT_SIZE: 100 / 200 / 300)

from sqlalchemy import text
from src.common.db import get_connection, set_search_path, DBConfig

cfg = DBConfig.from_env()

with get_connection() as conn:
    set_search_path(conn)
    conn.execute(text(f'TRUNCATE TABLE "{cfg.schema}"."segment_labels" RESTART IDENTITY;'))
    conn.commit()

print("🧹 Таблица segment_labels полностью очищена (ID сброшены).")

🧹 Таблица segment_labels полностью очищена (ID сброшены).


In [15]:
risk_stats = read_df("""
SELECT
    risk_label,
    COUNT(*) AS cnt
FROM segment_labels
GROUP BY risk_label
ORDER BY cnt DESC;
""")

display(risk_stats)

,risk_label,cnt
0,none,8617
1,flood,46


In [16]:
risk_check = read_df("""
SELECT
    COUNT(*) FILTER (WHERE risk_label IS NULL) AS is_null,
    COUNT(*) FILTER (WHERE risk_label = 'none') AS is_none,
    COUNT(*) AS total
FROM segment_labels;
""")

display(risk_check)

,is_null,is_none,total
0,0,8617,8663


In [17]:
null_risk = read_df("""
SELECT
    segment_id,
    track_id,
    segment_index,
    cluster_id,
    features
FROM segment_labels
WHERE risk_label IS NULL
LIMIT 20;
""")

display(null_risk)

,segment_id,track_id,segment_index,cluster_id,features


In [18]:
non_none_risk = read_df("""
SELECT
    segment_id,
    track_id,
    segment_index,
    cluster_id,
    risk_label,
    evac_label,
    features
FROM segment_labels
WHERE risk_label IS NOT NULL
  AND risk_label <> 'none'
LIMIT 20;
""")

display(non_none_risk)

,segment_id,track_id,segment_index,cluster_id,risk_label,evac_label,features
0,18,5,0,2,flood,easy,"{'ctx': {'ctx_id': 4, 'temp_c': 18.0, 'matched..."
1,19,5,1,0,flood,easy,"{'ctx': {'ctx_id': 4, 'temp_c': 18.0, 'matched..."
2,20,5,2,0,flood,easy,"{'ctx': {'ctx_id': 4, 'temp_c': 18.0, 'matched..."
3,21,5,3,0,flood,easy,"{'ctx': {'ctx_id': 4, 'temp_c': 18.0, 'matched..."
4,22,5,4,0,flood,easy,"{'ctx': {'ctx_id': 4, 'temp_c': 18.0, 'matched..."
5,615,8,59,0,flood,easy,"{'ctx': {'ctx_id': 30, 'temp_c': 1.0, 'matched..."
6,616,8,60,0,flood,easy,"{'ctx': {'ctx_id': 30, 'temp_c': 1.0, 'matched..."
7,646,8,90,0,flood,easy,"{'ctx': {'ctx_id': 31, 'temp_c': 2.1, 'matched..."
8,795,8,239,0,flood,easy,"{'ctx': {'ctx_id': 36, 'temp_c': 3.7, 'matched..."
9,796,8,240,0,flood,easy,"{'ctx': {'ctx_id': 36, 'temp_c': 3.7, 'matched..."


In [19]:
import json
import pandas as pd

sample = read_df("""
SELECT segment_id, features
FROM segment_labels
LIMIT 5;
""")

for _, r in sample.iterrows():
    print(f"\n📦 segment_id = {r['segment_id']}")
    print(json.dumps(r["features"], indent=2, ensure_ascii=False))


📦 segment_id = 4
{
  "ctx": {
    "ctx_id": 1,
    "temp_c": -6.2,
    "matched": true,
    "ctx_dist_m": 4.156429269978187,
    "nearby_amenity_total": 19.0,
    "nearby_highway_total": 157.0,
    "nearby_landuse_total": 14.0,
    "nearby_natural_total": 56.0,
    "nearby_building_total": 73.0,
    "nearby_waterway_total": 0.0
  },
  "slope": 4.3854984776862445,
  "points": 10,
  "distance_m": 12.823218746651952,
  "duration_s": 31416.054,
  "elev_max_m": 105.63411318034838,
  "elev_min_m": 49.39790688786853,
  "elev_gain_m": 111.15296891882275,
  "elev_loss_m": 74.9929992967798,
  "avg_speed_mps": 0.000408174073887572
}

📦 segment_id = 5
{
  "ctx": {
    "ctx_id": 1,
    "temp_c": -6.2,
    "matched": true,
    "ctx_dist_m": 2.5815973465947195,
    "nearby_amenity_total": 19.0,
    "nearby_highway_total": 157.0,
    "nearby_landuse_total": 14.0,
    "nearby_natural_total": 56.0,
    "nearby_building_total": 73.0,
    "nearby_waterway_total": 0.0
  },
  "slope": 4.532289263840481,
  

In [20]:
# =========================
# ЯЧЕЙКА 4 — МОДУЛЬ B (2.3): полуавтоматическая разметка RISK (flood/fire/none) + запись в БД
# =========================
# Идея: "модель разметки" = скоринг на основе геоданных (nearby + temp) + (опционально) cluster_id.
# Это нормально для 2.3: кластеризация уже сделана (ячейка 3), а разметка делается полуавтоматом.
#
# На соревнованиях ты меняешь только ПОРОГИ и ВЕСА, и (если нужно) логику сезонов.

import json
import numpy as np
import pandas as pd
from sqlalchemy import text

# -------------------------
# 0) НАСТРОЙКИ (их реально можно менять на соревнованиях)
# -------------------------

# Порог для присвоения риска (чем больше — тем "строже" ставим flood/fire)
FLOOD_THRESHOLD = 2.0
FIRE_THRESHOLD  = 2.0

# Весовые коэффициенты — "как модель" (скоринг)
W_WATERWAY = 2.0        # рядом вода/реки
W_WETLAND  = 1.5        # болото/влажные зоны (если появится)
W_LOW_SLOPE = 0.5       # плоско = более вероятно затопление

W_NATURAL  = 1.5        # лес/природа
W_LANDUSE_GRASS = 1.0   # травянистые территории (если есть)
W_HOT_TEMP = 1.5        # жарко
W_DRY_HINT = 0.5        # "косвенный" сухой намёк (если появится)

# Температурные диапазоны (можно подогнать)
SPRING_TEMP_MAX = 12.0  # весна условно до +12 (для flood)
SUMMER_TEMP_MIN = 22.0  # лето условно от +22 (для fire)

# Если контекста нет (ctx.matched=false), что делаем?
# True  -> разрешаем ставить риск по nearby (если nearby есть)
# False -> без ctx всегда risk=none
ALLOW_RISK_WITHOUT_TEMP = True


# -------------------------
# 1) Читаем сегменты
# -------------------------
seg = read_df("""
SELECT segment_id, track_id, segment_index, cluster_id, features, risk_label, evac_label
FROM segment_labels
ORDER BY segment_id;
""")

print("✅ Прочитали segment_labels:", seg.shape)
display(seg.head(3))

def as_dict(x):
    if x is None:
        return {}
    if isinstance(x, dict):
        return x
    if isinstance(x, str):
        try:
            return json.loads(x)
        except Exception:
            return {}
    return {}

seg["features_dict"] = seg["features"].apply(as_dict)

# -------------------------
# 2) Вытаскиваем нужные поля из features->ctx
# -------------------------
def get_ctx_val(fd: dict, key: str, default=None):
    ctx = fd.get("ctx", {}) if isinstance(fd, dict) else {}
    return ctx.get(key, default)

def safe_float(v):
    try:
        if v is None:
            return np.nan
        return float(v)
    except Exception:
        return np.nan

def safe_int(v):
    try:
        if v is None:
            return 0
        return int(v)
    except Exception:
        return 0

seg["temp_c"] = seg["features_dict"].apply(lambda d: safe_float(get_ctx_val(d, "temp_c", None)))
seg["ctx_matched"] = seg["features_dict"].apply(lambda d: bool(get_ctx_val(d, "matched", False)))

seg["nearby_waterway_total"] = seg["features_dict"].apply(lambda d: safe_float(get_ctx_val(d, "nearby_waterway_total", 0.0)))
seg["nearby_natural_total"]  = seg["features_dict"].apply(lambda d: safe_float(get_ctx_val(d, "nearby_natural_total", 0.0)))
seg["nearby_landuse_total"]  = seg["features_dict"].apply(lambda d: safe_float(get_ctx_val(d, "nearby_landuse_total", 0.0)))

# сегментные признаки (которые мы уже считали)
seg["slope"] = seg["features_dict"].apply(lambda d: safe_float(d.get("slope", np.nan)))
seg["distance_m"] = seg["features_dict"].apply(lambda d: safe_float(d.get("distance_m", np.nan)))

# -------------------------
# 3) Скоринговая "модель" риска
# -------------------------
def flood_score(row):
    # основа: рядом вода + (весенний диапазон температуры) + (плоский рельеф)
    score = 0.0
    
    waterway = row["nearby_waterway_total"]
    if waterway > 0:
        score += W_WATERWAY
    
    # если температура известна
    t = row["temp_c"]
    if np.isfinite(t):
        # "весна": не жарко
        if t <= SPRING_TEMP_MAX:
            score += 1.0
    else:
        # если темпы нет — разрешаем риск без температуры
        if ALLOW_RISK_WITHOUT_TEMP:
            score += 0.3  # слабый бонус
    
    # плоско = риск затопления чуть выше
    sl = row["slope"]
    if np.isfinite(sl) and sl < 0.01:
        score += W_LOW_SLOPE
    
    return score

def fire_score(row):
    # основа: много природы/ландшафта + жарко (если есть температура)
    score = 0.0
    
    nat = row["nearby_natural_total"]
    land = row["nearby_landuse_total"]
    
    # если рядом "природы" много
    if nat >= 10:
        score += W_NATURAL
    
    # landuse_total у нас агрегат, но можно дать небольшой вклад
    if land >= 5:
        score += W_LANDUSE_GRASS
    
    t = row["temp_c"]
    if np.isfinite(t):
        if t >= SUMMER_TEMP_MIN:
            score += W_HOT_TEMP
    else:
        if ALLOW_RISK_WITHOUT_TEMP:
            score += 0.2  # слабый бонус (без температуры осторожно)
    
    return score

seg["flood_score"] = seg.apply(flood_score, axis=1)
seg["fire_score"]  = seg.apply(fire_score, axis=1)

def choose_risk(row):
    fs = row["flood_score"]
    fir = row["fire_score"]
    
    # если вообще нет ctx и мы запрещаем разметку без темпы -> none
    if (not row["ctx_matched"]) and (not ALLOW_RISK_WITHOUT_TEMP):
        return "none", "no_ctx"
    
    # выбор максимума при превышении порога
    if fs >= FLOOD_THRESHOLD and fs >= fir:
        return "flood", f"waterway={row['nearby_waterway_total']}, temp={row['temp_c']}"
    if fir >= FIRE_THRESHOLD and fir > fs:
        return "fire", f"natural={row['nearby_natural_total']}, temp={row['temp_c']}"
    
    return "none", "below_threshold"

chosen = seg.apply(choose_risk, axis=1, result_type="expand")
seg["risk_label_new"] = chosen[0]
seg["risk_reason"]    = chosen[1]

print("\n📊 risk_label_new распределение:")
display(seg["risk_label_new"].value_counts(dropna=False).to_frame("cnt"))

# -------------------------
# 4) Вшиваем объяснение в features (чтобы преподавателю было видно "почему")
# -------------------------
def update_features(fd: dict, row) -> dict:
    if not isinstance(fd, dict):
        fd = {}
    fd = dict(fd)  # копия
    
    fd["risk_model"] = {
        "version": "v1-score",
        "flood_score": float(row["flood_score"]),
        "fire_score": float(row["fire_score"]),
        "risk_label": row["risk_label_new"],
        "reason": row["risk_reason"],
        "thresholds": {"flood": FLOOD_THRESHOLD, "fire": FIRE_THRESHOLD},
    }
    return fd

seg["features_updated"] = seg.apply(lambda r: update_features(r["features_dict"], r), axis=1)

def to_json_str(obj) -> str:
    return json.dumps(obj, ensure_ascii=False)

# -------------------------
# 5) Записываем risk_label + features обратно в БД
#    ВАЖНО: используем CAST(:features AS jsonb) — чтобы не было ошибки ":features::jsonb"
# -------------------------
upd_sql = f"""
UPDATE "{cfg.schema}"."segment_labels"
SET
    risk_label = :risk_label,
    features   = CAST(:features AS jsonb)
WHERE segment_id = :segment_id;
"""

rows_updated = 0
with get_connection() as conn:
    set_search_path(conn)
    for _, r in seg.iterrows():
        conn.execute(text(upd_sql), {
            "segment_id": int(r["segment_id"]),
            "risk_label": r["risk_label_new"],
            "features": to_json_str(r["features_updated"]),
        })
        rows_updated += 1
    conn.commit()

print("\n✅ Записали risk_label + features в БД. Обновлено строк:", rows_updated)

# -------------------------
# 6) Контроль из БД
# -------------------------
risk_dist = read_df("""
SELECT risk_label, COUNT(*) AS cnt
FROM segment_labels
GROUP BY risk_label
ORDER BY cnt DESC;
""")
print("\n📊 risk_label распределение (из БД):")
display(risk_dist)

# покажем примеры flood/fire (если появились)
examples = read_df("""
SELECT segment_id, track_id, segment_index, cluster_id, risk_label, evac_label, features
FROM segment_labels
WHERE risk_label IN ('flood','fire')
ORDER BY segment_id
LIMIT 10;
""")
print("\n🔎 Примеры сегментов с риском:")
display(examples)

✅ Прочитали segment_labels: (8663, 7)


,segment_id,track_id,segment_index,cluster_id,features,risk_label,evac_label
0,1,2,0,4,"{'ctx': {'ctx_id': 1, 'temp_c': -6.2, 'matched...",none,hard
1,2,2,1,4,"{'ctx': {'ctx_id': 1, 'temp_c': -6.2, 'matched...",none,easy
2,3,2,2,2,"{'ctx': {'ctx_id': 1, 'temp_c': -6.2, 'matched...",none,medium



📊 risk_label_new распределение:


,cnt
risk_label_new,
none,8427
flood,225
fire,11



✅ Записали risk_label + features в БД. Обновлено строк: 8663

📊 risk_label распределение (из БД):


,risk_label,cnt
0,none,8427
1,flood,225
2,fire,11



🔎 Примеры сегментов с риском:


,segment_id,track_id,segment_index,cluster_id,risk_label,evac_label,features
0,1,2,0,4,fire,hard,"{'ctx': {'ctx_id': 1, 'temp_c': -6.2, 'matched..."
1,2,2,1,4,fire,easy,"{'ctx': {'ctx_id': 1, 'temp_c': -6.2, 'matched..."
2,3,2,2,2,fire,medium,"{'ctx': {'ctx_id': 1, 'temp_c': -6.2, 'matched..."
3,4,2,3,4,fire,hard,"{'ctx': {'ctx_id': 1, 'temp_c': -6.2, 'matched..."
4,5,2,4,3,fire,hard,"{'ctx': {'ctx_id': 1, 'temp_c': -6.2, 'matched..."
5,6,3,0,2,flood,medium,"{'ctx': {'ctx_id': 2, 'temp_c': 8.5, 'matched'..."
6,7,3,1,2,flood,medium,"{'ctx': {'ctx_id': 2, 'temp_c': 8.5, 'matched'..."
7,8,3,2,2,flood,easy,"{'ctx': {'ctx_id': 2, 'temp_c': 8.5, 'matched'..."
8,9,3,3,2,flood,easy,"{'ctx': {'ctx_id': 2, 'temp_c': 8.5, 'matched'..."
9,10,3,4,2,flood,medium,"{'ctx': {'ctx_id': 2, 'temp_c': 8.5, 'matched'..."


In [21]:
# =========================
# ЯЧЕЙКА 5 — МОДУЛЬ B (2.4): оценка качества разметки + качество кластеров
# =========================
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# 1) Считаем из БД итоговую таблицу сегментов
df = read_df("""
SELECT segment_id, track_id, segment_index, cluster_id, risk_label, evac_label, features
FROM segment_labels
ORDER BY segment_id;
""")

# 2) Распакуем features (jsonb) -> dict
import json
def to_dict(x):
    if x is None:
        return {}
    if isinstance(x, dict):
        return x
    if isinstance(x, str):
        try:
            return json.loads(x)
        except Exception:
            return {}
    return {}

df["f"] = df["features"].apply(to_dict)

# 3) Достаём те же фичи, что использовали для кластеризации
SEG_FEATURES = ["distance_m","duration_s","avg_speed_mps","elev_gain_m","elev_loss_m","elev_min_m","elev_max_m","slope","points"]

for k in SEG_FEATURES:
    df[k] = df["f"].apply(lambda d: d.get(k, np.nan))

X = df[SEG_FEATURES].copy()

# 4) Подготовка (импут + скейл)
prep = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])
X_prep = prep.fit_transform(X)

# 5) Silhouette по уже присвоенным кластерам (из БД)
labels = df["cluster_id"].astype(int).values
sil = silhouette_score(X_prep, labels) if len(np.unique(labels)) > 1 else np.nan
print("✅ Silhouette (качество кластеров):", round(float(sil), 4))

# 6) Распределение кластеров
cl_dist = df["cluster_id"].value_counts().sort_index().to_frame("cnt")
cl_dist["share_%"] = (100 * cl_dist["cnt"] / len(df)).round(2)
print("\n📊 cluster_id распределение:")
display(cl_dist)

# 7) Распределение меток разметки
risk_dist = df["risk_label"].value_counts().to_frame("cnt")
risk_dist["share_%"] = (100 * risk_dist["cnt"] / len(df)).round(2)
print("\n📊 risk_label распределение:")
display(risk_dist)

evac_dist = df["evac_label"].value_counts().to_frame("cnt")
evac_dist["share_%"] = (100 * evac_dist["cnt"] / len(df)).round(2)
print("\n📊 evac_label распределение:")
display(evac_dist)

# 8) Профили кластеров (средние значения признаков)
profile = df.groupby("cluster_id")[SEG_FEATURES].mean(numeric_only=True)
profile["count"] = df.groupby("cluster_id")["segment_id"].count()
print("\n🧠 Профили кластеров (средние + count):")
display(profile)

# 9) Быстрая 2D-проекция PCA для “визуального анализа кластеров”
pca = PCA(n_components=2, random_state=42)
XY = pca.fit_transform(X_prep)
df_vis = pd.DataFrame({"pc1": XY[:,0], "pc2": XY[:,1], "cluster_id": labels})
print("\nℹ️ PCA готово: df_vis (pc1/pc2/cluster_id).")
display(df_vis.head(5))

print("\n💡 Готовность к модулю В:")
print("- Модуль В будет учить модель предсказывать risk_label и evac_label по признакам сегмента.")
print("- Если классы риска слишком редкие (например fire=0), модель будет учиться плохо → надо смягчить правила разметки.")

✅ Silhouette (качество кластеров): 0.6072

📊 cluster_id распределение:


,cnt,share_%
cluster_id,,
0,7300,84.27
1,1011,11.67
2,323,3.73
3,26,0.30
4,3,0.03



📊 risk_label распределение:


,cnt,share_%
risk_label,,
none,8427,97.28
flood,225,2.60
fire,11,0.13



📊 evac_label распределение:


...


🧠 Профили кластеров (средние + count):


,distance_m,duration_s,avg_speed_mps,elev_gain_m,elev_loss_m,elev_min_m,elev_max_m,slope,points,count
cluster_id,,,,,,,,,,
0,773.483880,214.443561,4.027477,3.168499,3.237168,181.478010,185.566900,0.016823,9.998630,7300
1,574.466489,288.205128,3.848655,10.160438,10.528826,784.234889,798.832038,0.039608,9.997033,1011
2,16.883285,22.954789,0.861258,1.292205,1.328886,144.362650,146.297372,0.254031,9.990712,323
3,324.384208,437.756200,2.662884,3.054142,7.147264,218.482353,224.937157,0.478432,3.961538,26
4,470.996725,25175.189000,0.016734,109.231017,93.293514,37.430863,101.776093,3.356677,10.000000,3



ℹ️ PCA готово: df_vis (pc1/pc2/cluster_id).


,pc1,pc2,cluster_id
0,20.767655,46.323377,4
1,17.336335,35.888118,4
2,6.387936,9.586975,2
3,23.322372,55.884487,4
4,7.247442,24.661216,3



💡 Готовность к модулю В:
- Модуль В будет учить модель предсказывать risk_label и evac_label по признакам сегмента.
- Если классы риска слишком редкие (например fire=0), модель будет учиться плохо → надо смягчить правила разметки.
